# IF3070 Foundations of Artificial Intelligence | Tugas Besar 2


Group Number: 38

Group Members:
- Alessandro Jusack H.      (18222025)
- Viktor Arsindiantoro      (18222083)
- Matthew Nicholas Gunawan  (18222058)
- Daffa Ramadhan Elengi     (1822009)

## Import Libraries

In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as stats
from sklearn.preprocessing import LabelEncoder

## Import Dataset

In [389]:
# df = pd.read_csv('https://drive.google.com/uc?id=1ZUtiaty9RPXhpz5F2Sy3dFPHF4YIt5iU')
df = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')
# df.head()

# 1. Split Training Set and Validation Set

Splitting the training and validation set works as an early diagnostic towards the performance of the model we train. This is done before the preprocessing steps to **avoid data leakage inbetween the sets**. If you want to use k-fold cross-validation, split the data later and do the cleaning and preprocessing separately for each split.

Note: For training, you should use the data contained in the `train` folder given by the TA. The `test` data is only used for kaggle submission.

In [390]:
from sklearn.model_selection import train_test_split

train_set, val_set = train_test_split(df, stratify=df['label'], test_size=0.2, random_state=42)
train_set_original = train_set.copy()

In [391]:
# import math
# train_set_filtered = train_set.select_dtypes(exclude=['object']).copy()
# train_set_filtered = train_set_filtered.drop(columns="id")
# numeric_cols = [col for col in train_set_filtered.columns if col != 'label']
# n_cols = 2  
# n_rows = math.ceil(len(numeric_cols) / n_cols)

# fig, axes = plt.subplots(n_rows, n_cols, figsize=(17, 5 * n_rows), squeeze=False)

# for idx, col in enumerate(numeric_cols):

#     row = idx // n_cols
#     colpos = idx % n_cols
    
#     ax = axes[row][colpos]  

#     sns.boxplot(
#         data=train_set, 
#         x="label", 
#         y=col, 
#         hue="label", 
#         palette="Set2", 
#         ax=ax
#     )
#     ax.set_title(f"Distribution of '{col}' by label")
    

#     ax.get_legend().remove()


# for j in range(idx+1, n_rows * n_cols):
#     row = j // n_cols
#     colpos = j % n_cols
#     axes[row][colpos].set_visible(False)

# plt.tight_layout()
# plt.show()
        
    

# 2. Data Cleaning and Preprocessing

This step is the first thing to be done once a Data Scientist have grasped a general knowledge of the data. Raw data is **seldom ready for training**, therefore steps need to be taken to clean and format the data for the Machine Learning model to interpret.

By performing data cleaning and preprocessing, you ensure that your dataset is ready for model training, leading to more accurate and reliable machine learning results. These steps are essential for transforming raw data into a format that machine learning algorithms can effectively learn from and make predictions.

We will give some common methods for you to try, but you only have to **at least implement one method for each process**. For each step that you will do, **please explain the reason why did you do that process. Write it in a markdown cell under the code cell you wrote.**

## A. Data Cleaning

**Data cleaning** is the crucial first step in preparing your dataset for machine learning. Raw data collected from various sources is often messy and may contain errors, missing values, and inconsistencies. Data cleaning involves the following steps:

1. **Handling Missing Data:** Identify and address missing values in the dataset. This can include imputing missing values, removing rows or columns with excessive missing data, or using more advanced techniques like interpolation.

2. **Dealing with Outliers:** Identify and handle outliers, which are data points significantly different from the rest of the dataset. Outliers can be removed or transformed to improve model performance.

3. **Data Validation:** Check for data integrity and consistency. Ensure that data types are correct, categorical variables have consistent labels, and numerical values fall within expected ranges.

4. **Removing Duplicates:** Identify and remove duplicate rows, as they can skew the model's training process and evaluation metrics.

5. **Feature Engineering**: Create new features or modify existing ones to extract relevant information. This step can involve scaling, normalizing, or encoding features for better model interpretability.

### I. Handling Missing Data

Missing data can adversely affect the performance and accuracy of machine learning models. There are several strategies to handle missing data in machine learning:

1. **Data Imputation:**

    a. **Mean, Median, or Mode Imputation:** For numerical features, you can replace missing values with the mean, median, or mode of the non-missing values in the same feature. This method is simple and often effective when data is missing at random.

    b. **Constant Value Imputation:** You can replace missing values with a predefined constant value (e.g., 0) if it makes sense for your dataset and problem.

    c. **Imputation Using Predictive Models:** More advanced techniques involve using predictive models to estimate missing values. For example, you can train a regression model to predict missing numerical values or a classification model to predict missing categorical values.

2. **Deletion of Missing Data:**

    a. **Listwise Deletion:** In cases where the amount of missing data is relatively small, you can simply remove rows with missing values from your dataset. However, this approach can lead to a loss of valuable information.

    b. **Column (Feature) Deletion:** If a feature has a large number of missing values and is not critical for your analysis, you can consider removing that feature altogether.

3. **Domain-Specific Strategies:**

    a. **Domain Knowledge:** In some cases, domain knowledge can guide the imputation process. For example, if you know that missing values are related to a specific condition, you can impute them accordingly.

4. **Imputation Libraries:**

    a. **Scikit-Learn:** Scikit-Learn provides a `SimpleImputer` class that can handle basic imputation strategies like mean, median, and mode imputation.

    b. **Fancyimpute:** Fancyimpute is a Python library that offers more advanced imputation techniques, including matrix factorization, k-nearest neighbors, and deep learning-based methods.

The choice of imputation method should be guided by the nature of your data, the amount of missing data, the problem you are trying to solve, and the assumptions you are willing to make.

In [392]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.impute import SimpleImputer

class FeatureImputer(BaseEstimator, TransformerMixin):
    def __init__(self,
                 att=None,
                 label_col="label"):
        # ? manual input
        if att is None:
            att = [
                "IsDomainIP", "HasObfuscation", "IsHTTPS", "HasTitle", 
                "HasFavicon", "Robots", "IsResponsive", "HasDescription", 
                "HasExternalFormSubmit", "HasSocialNet", "HasSubmitButton", 
                "HasHiddenFields", "HasPasswordField", "Pay", "Bank", "Crypto", 
                "HasCopyrightInfo"
            ]
        self.att = att
        self.label_col = label_col
        
        self.cat_imputer_att = SimpleImputer(strategy='most_frequent')
        self.cat_imputer_general = SimpleImputer(strategy='most_frequent')
        self.num_imputer = SimpleImputer(strategy='mean')

    def fit(self, X, y=None):
        X = pd.DataFrame(X).copy()

        numeric_cols = X.select_dtypes(include=['float64', 'int64']).columns.tolist()
        categorical_cols = X.select_dtypes(include=['object']).columns.tolist()

        self.numeric_cols_ = [
            col for col in numeric_cols
            if (col not in self.att) and (col != self.label_col)
        ]
        self.cat_cols_general_ = [
            col for col in categorical_cols
            if col not in self.att
        ]
        
        att_cols_present = [c for c in self.att if c in X.columns]
        if att_cols_present:
            self.cat_imputer_att.fit(X[att_cols_present])
        
        if self.numeric_cols_:
            self.num_imputer.fit(X[self.numeric_cols_])
            
        if self.cat_cols_general_:
            self.cat_imputer_general.fit(X[self.cat_cols_general_])

        return self

    def transform(self, X):
        X = pd.DataFrame(X).copy()
        
        #? att column dari vik
        att_cols_present = [c for c in self.att if c in X.columns]
        if att_cols_present:
            X[att_cols_present] = self.cat_imputer_att.transform(X[att_cols_present])
        
        #? regular numeridcal  columns
        if self.numeric_cols_:
            X[self.numeric_cols_] = self.num_imputer.transform(X[self.numeric_cols_])
        
        #? else categorical columns
        if self.cat_cols_general_:
            cat_cols_present = [c for c in self.cat_cols_general_ if c in X.columns]
            if cat_cols_present:
                X[cat_cols_present] = self.cat_imputer_general.transform(X[cat_cols_present])
        
        return X


### II. Dealing with Outliers

Outliers are data points that significantly differ from the majority of the data. They can be unusually high or low values that do not fit the pattern of the rest of the dataset. Outliers can significantly impact model performance, so it is important to handle them properly.

Some methods to handle outliers:
1. **Imputation**: Replace with mean, median, or a boundary value.
2. **Clipping**: Cap values to upper and lower limits.
3. **Transformation**: Use log, square root, or power transformations to reduce their influence.
4. **Model-Based**: Use algorithms robust to outliers (e.g., tree-based models, Huber regression).

In [393]:
import numpy as np
import pandas as pd
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import PowerTransformer

class FeatureTransformator(BaseEstimator, TransformerMixin):
    def __init__(self, skew_threshold=1.0, target_column='label'):

        self.skew_threshold = skew_threshold
        self.target_column = target_column
        
        self.log1p_cols_ = []
        self.power_transformers_ = {}

    def fit(self, X, y=None):
        X = pd.DataFrame(X).copy()
        for col in X.select_dtypes(include=[np.number]).columns:
            if col == self.target_column:
                continue

            skewness = X[col].skew()
            if abs(skewness) > self.skew_threshold:
                if skewness > 0:
                    self.log1p_cols_.append(col)
                else:
                    pt = PowerTransformer(method='yeo-johnson')
                    
                    pt.fit(X[col].values.reshape(-1, 1))
                    
                    self.power_transformers_[col] = pt
        
        return self

    def transform(self, X):
        X = pd.DataFrame(X).copy()
        
        for col in self.log1p_cols_:
            if col in X.columns:
                X[col] = np.log1p(X[col])

        for col, pt in self.power_transformers_.items():
            if col in X.columns:
                X[col] = pt.transform(X[col].values.reshape(-1, 1)).flatten()

        return X


### III. Remove Duplicates
Handling duplicate values is crucial because they can compromise data integrity, leading to inaccurate analysis and insights. Duplicate entries can bias machine learning models, causing overfitting and reducing their ability to generalize to new data. They also inflate the dataset size unnecessarily, increasing computational costs and processing times. Additionally, duplicates can distort statistical measures and lead to inconsistencies, ultimately affecting the reliability of data-driven decisions and reporting. Ensuring data quality by removing duplicates is essential for accurate, efficient, and consistent analysis.

In [394]:
import pandas as pd
from sklearn.base import BaseEstimator, TransformerMixin

class DuplicateRemover(BaseEstimator, TransformerMixin):
    def __init__(self, verbose=False):
        self.verbose = verbose

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X = X.copy()
        
        before_len = len(X)
        X.drop_duplicates(inplace=True)
        after_len = len(X)
        X.reset_index(drop=True, inplace=True)

        if self.verbose:
            print(f"Before dropping duplicates: {before_len}")
            print(f"After dropping duplicates:  {after_len}")

        return X


### IV. Feature Engineering

**Feature engineering** involves creating new features (input variables) or transforming existing ones to improve the performance of machine learning models. Feature engineering aims to enhance the model's ability to learn patterns and make accurate predictions from the data. It's often said that "good features make good models."

1. **Feature Selection:** Feature engineering can involve selecting the most relevant and informative features from the dataset. Removing irrelevant or redundant features not only simplifies the model but also reduces the risk of overfitting.

2. **Creating New Features:** Sometimes, the existing features may not capture the underlying patterns effectively. In such cases, engineers create new features that provide additional information. For example:
   
   - **Polynomial Features:** Engineers may create new features by taking the square, cube, or other higher-order terms of existing numerical features. This can help capture nonlinear relationships.
   
   - **Interaction Features:** Interaction features are created by combining two or more existing features. For example, if you have features "length" and "width," you can create an "area" feature by multiplying them.

3. **Binning or Discretization:** Continuous numerical features can be divided into bins or categories. For instance, age values can be grouped into bins like "child," "adult," and "senior."

4. **Domain-Specific Feature Engineering:** Depending on the domain and problem, engineers may create domain-specific features. For example, in fraud detection, features related to transaction history and user behavior may be engineered to identify anomalies.

Feature engineering is both a creative and iterative process. It requires a deep understanding of the data, domain knowledge, and experimentation to determine which features will enhance the model's predictive power.

In [395]:
import numpy as np
import pandas as pd
from sklearn.base import BaseEstimator, TransformerMixin

class FeatureCreator(BaseEstimator, TransformerMixin):
    def __init__(self, domain_weight=0.6, url_weight=0.4):
        self.domain_weight = domain_weight
        self.url_weight = url_weight

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X = X.copy()
        
        #? total resources
        X['resources'] = (
            X['NoOfImage'] + X['NoOfCSS'] + X['NoOfJS'] +
            X['NoOfSelfRef'] + X['NoOfEmptyRef'] + X['NoOfExternalRef']
        )

        #? feature securing stuffs
        X['IsHTTPS'] = X['IsHTTPS'].astype(int)
        X['Robots']  = X['Robots'].astype(int)

        conditions = [
            (X['IsHTTPS'] == 1) & (X['Robots'] == 1),
            (X['IsHTTPS'] == 1) & (X['Robots'] == 0),
            (X['IsHTTPS'] == 0) & (X['Robots'] == 1),
            (X['IsHTTPS'] == 0) & (X['Robots'] == 0),
        ]
        risks = [3, 2, 1, 0]
        X['isHighRisk'] = np.select(conditions, risks)

        # ? non-phishing features
        X['Non_Phising'] = (
            X['IsResponsive'] + 
            X['HasDescription'] + 
            X['HasSocialNet'] + 
            X['HasCopyrightInfo']
        )

        #? weighted score
        X['WeightedScore'] = (
            self.domain_weight * X['DomainTitleMatchScore'] + 
            self.url_weight * X['URLTitleMatchScore']
        )

        #? digit and special char ratio
        X['DigitAndSpecialCharRatio'] = (
            (X['NoOfDegitsInURL'] + X['NoOfOtherSpecialCharsInURL']) / X['URLLength']
        )

        #? digit ratio by special char ratio
        X['digit_special_mult'] = (
            X['DegitRatioInURL'] * X['SpacialCharRatioInURL']
        )

        #? is_https_line_interaction
        X['is_https_line_interaction'] = (
            X['IsHTTPS'] * np.log(X['LineOfCode'] + 1)
        )

        return X


In [396]:
import pandas as pd
from sklearn.base import BaseEstimator, TransformerMixin

class FeatureSelector(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.columns_to_drop = [
            "id", "IsDomainIP", "TLDLength", "NoOfSubDomain", "HasObfuscation",
            "NoOfObfuscatedChar", "ObfuscationRatio", "NoOfAmpersandInURL",
            "NoOfURLRedirect", "NoOfSelfRedirect", "NoOfPopup", "HasExternalFormSubmit",
            "HasPasswordField", "Bank", "Crypto", "TLDLegitimateProb", "DomainLength",
            "NoOfEqualsInURL", "NoOfQMarkInURL", "FILENAME","URL", "Domain", "Title"
        ]
    
    def fit(self, X, y=None):
    
        return self

    def transform(self, X):
        return X.drop(columns=self.columns_to_drop, axis=1)


In [397]:
import pandas as pd
from sklearn.base import BaseEstimator, TransformerMixin

class FeatureObjectDropper(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X = X.copy()
        object_cols = X.select_dtypes(include=['object']).columns
        
        X = X.drop(columns=object_cols, axis=1)
        
        return X


## B. Data Preprocessing

**Data preprocessing** is a broader step that encompasses both data cleaning and additional transformations to make the data suitable for machine learning algorithms. Its primary goals are:

1. **Feature Scaling:** Ensure that numerical features have similar scales. Common techniques include Min-Max scaling (scaling to a specific range) or standardization (mean-centered, unit variance).

2. **Encoding Categorical Variables:** Machine learning models typically work with numerical data, so categorical variables need to be encoded. This can be done using one-hot encoding, label encoding, or more advanced methods like target encoding.

3. **Handling Imbalanced Classes:** If dealing with imbalanced classes in a binary classification task, apply techniques such as oversampling, undersampling, or using different evaluation metrics to address class imbalance.

4. **Dimensionality Reduction:** Reduce the number of features using techniques like Principal Component Analysis (PCA) or feature selection to simplify the model and potentially improve its performance.

5. **Normalization:** Normalize data to achieve a standard distribution. This is particularly important for algorithms that assume normally distributed data.

### I. Feature Scaling

**Feature scaling** is a preprocessing technique used in machine learning to standardize the range of independent variables or features of data. The primary goal of feature scaling is to ensure that all features contribute equally to the training process and that machine learning algorithms can work effectively with the data.

Here are the main reasons why feature scaling is important:

1. **Algorithm Sensitivity:** Many machine learning algorithms are sensitive to the scale of input features. If the scales of features are significantly different, some algorithms may perform poorly or take much longer to converge.

2. **Distance-Based Algorithms:** Algorithms that rely on distances or similarities between data points, such as k-nearest neighbors (KNN) and support vector machines (SVM), can be influenced by feature scales. Features with larger scales may dominate the distance calculations.

3. **Regularization:** Regularization techniques, like L1 (Lasso) and L2 (Ridge) regularization, add penalty terms based on feature coefficients. Scaling ensures that all features are treated equally in the regularization process.

Common methods for feature scaling include:

1. **Min-Max Scaling (Normalization):** This method scales features to a specific range, typically [0, 1]. It's done using the following formula:

   $$X' = \frac{X - X_{min}}{X_{max} - X_{min}}$$

   - Here, $X$ is the original feature value, $X_{min}$ is the minimum value of the feature, and $X_{max}$ is the maximum value of the feature.  
<br />
<br />
2. **Standardization (Z-score Scaling):** This method scales features to have a mean (average) of 0 and a standard deviation of 1. It's done using the following formula:

   $$X' = \frac{X - \mu}{\sigma}$$

   - $X$ is the original feature value, $\mu$ is the mean of the feature, and $\sigma$ is the standard deviation of the feature.  
<br />
<br />
3. **Robust Scaling:** Robust scaling is a method that scales features to the interquartile range (IQR) and is less affected by outliers. It's calculated as:

   $$X' = \frac{X - Q1}{Q3 - Q1}$$

   - $X$ is the original feature value, $Q1$ is the first quartile (25th percentile), and $Q3$ is the third quartile (75th percentile) of the feature.  
<br />
<br />
4. **Log Transformation:** In cases where data is highly skewed or has a heavy-tailed distribution, taking the logarithm of the feature values can help stabilize the variance and improve scaling.

The choice of scaling method depends on the characteristics of your data and the requirements of your machine learning algorithm. **Min-max scaling and standardization are the most commonly used techniques and work well for many datasets.**

Scaling should be applied separately to each training and test set to prevent data leakage from the test set into the training set. Additionally, **some algorithms may not require feature scaling, particularly tree-based models.**

In [398]:
import pandas as pd
import numpy as np
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import RobustScaler

class FeatureScaler(BaseEstimator, TransformerMixin):
    def __init__(self, columns=None, 
                 outlier_threshold=5.0, 
                 label_col='label'):
        self.columns = columns
        self.outlier_threshold = outlier_threshold
        self.label_col = label_col
        self.scaler = RobustScaler()
        
    def _check_outliers_iqr(self, df, col):
        col_Q1 = df[col].quantile(0.25)
        col_Q3 = df[col].quantile(0.75)
        col_IQR = col_Q3 - col_Q1
        
        col_RLB = col_Q1 - 1.5 * col_IQR 
        col_RUB = col_Q3 + 1.5 * col_IQR  
        
        outliers_count = df.loc[(df[col] < col_RLB) | (df[col] > col_RUB), col].shape[0]
        outlier_percent = outliers_count / len(df) * 100.0
        
        return outlier_percent
    
    def fit(self, X, y=None):
        X = pd.DataFrame(X).copy()

        if self.columns is not None:
            self.columns_ = self.columns
        
        else:
            numeric_cols = X.select_dtypes(include=['float64', 'int64']).columns.tolist()
            numeric_cols = [col for col in numeric_cols if col != self.label_col]
            
            self.columns_ = []
            for col in numeric_cols:
                outlier_percent = self._check_outliers_iqr(X, col)
                if outlier_percent > self.outlier_threshold:
                    self.columns_.append(col)
        
        if self.columns_:
            self.scaler.fit(X[self.columns_])
        
        return self

    def transform(self, X):
        X = pd.DataFrame(X).copy()
        
        if self.columns_:
            X[self.columns_] = self.scaler.transform(X[self.columns_])
        
        return X


### II. Feature Encoding

**Feature encoding**, also known as **categorical encoding**, is the process of converting categorical data (non-numeric data) into a numerical format so that it can be used as input for machine learning algorithms. Most machine learning models require numerical data for training and prediction, so feature encoding is a critical step in data preprocessing.

Categorical data can take various forms, including:

1. **Nominal Data:** Categories with no intrinsic order, like colors or country names.  

2. **Ordinal Data:** Categories with a meaningful order but not necessarily equidistant, like education levels (e.g., "high school," "bachelor's," "master's").

There are several common methods for encoding categorical data:

1. **Label Encoding:**

   - Label encoding assigns a unique integer to each category in a feature.
   - It's suitable for ordinal data where there's a clear order among categories.
   - For example, if you have an "education" feature with values "high school," "bachelor's," and "master's," you can encode them as 0, 1, and 2, respectively.
<br />
<br />
2. **One-Hot Encoding:**

   - One-hot encoding creates a binary (0 or 1) column for each category in a nominal feature.
   - It's suitable for nominal data where there's no inherent order among categories.
   - Each category becomes a new feature, and the presence (1) or absence (0) of a category is indicated for each row.
<br />
<br />
3. **Target Encoding (Mean Encoding):**

   - Target encoding replaces each category with the mean of the target variable for that category.
   - It's often used for classification problems.

In [399]:
import pandas as pd
from sklearn.base import BaseEstimator, TransformerMixin

class FeatureEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, 
                 top_n=4, 
                 target_column='TLD'):
        self.top_n = top_n
        self.target_column = target_column

    def fit(self, X, y=None):
        X = pd.DataFrame(X).copy()

        if self.target_column in X.columns:
            counts = X[self.target_column].value_counts()
            self.top_values_ = set(counts.index[:self.top_n])
        else:
            self.top_values_ = set()

        return self

    def transform(self, X):
        """
        1) Lump values outside the top N into 'else' for the target_column.
        2) One-hot encode all object columns.
        """
        X_encoded = pd.DataFrame(X).copy()

        if self.target_column in X_encoded.columns:
            X_encoded[self.target_column] = X_encoded[self.target_column].apply(
                lambda x: x if x in self.top_values_ else "else"
            )

        cat_cols = X_encoded.select_dtypes(include=['object']).columns

        for col in cat_cols:
            one_hot = pd.get_dummies(X_encoded[col], prefix=col, dtype=int)
            X_encoded = pd.concat([X_encoded, one_hot], axis=1)
            X_encoded.drop(col, axis=1, inplace=True)

        return X_encoded


### III. Handling Imbalanced Dataset

**Handling imbalanced datasets** is important because imbalanced data can lead to several issues that negatively impact the performance and reliability of machine learning models. Here are some key reasons:

1. **Biased Model Performance**:

 - Models trained on imbalanced data tend to be biased towards the majority class, leading to poor performance on the minority class. This can result in misleading accuracy metrics.

2. **Misleading Accuracy**:

 - High overall accuracy can be misleading in imbalanced datasets. For example, if 95% of the data belongs to one class, a model that always predicts the majority class will have 95% accuracy but will fail to identify the minority class.

3. **Poor Generalization**:

 - Models trained on imbalanced data may not generalize well to new, unseen data, especially if the minority class is underrepresented.


Some methods to handle imbalanced datasets:
1. **Resampling Methods**:

 - Oversampling: Increase the number of instances in the minority class by duplicating or generating synthetic samples (e.g., SMOTE).
 - Undersampling: Reduce the number of instances in the majority class to balance the dataset.

2. **Evaluation Metrics**:

 - Use appropriate evaluation metrics such as precision, recall, F1-score, ROC-AUC, and confusion matrix instead of accuracy to better assess model performance on imbalanced data.

3. **Algorithmic Approaches**:

 - Use algorithms that are designed to handle imbalanced data, such as decision trees, random forests, or ensemble methods.
 - Adjust class weights in algorithms to give more importance to the minority class.

In [400]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, f1_score, recall_score, cohen_kappa_score, balanced_accuracy_score, classification_report
from imblearn.over_sampling import SMOTE
from imblearn.combine import SMOTEENN
from imblearn.combine import SMOTETomek

In [401]:
def my_smote(X_before, y_before):
    smote = SMOTE(random_state=42)
    X_after, y_after = smote.fit_resample(X_before, y_before)
    return X_after, y_after

def my_smotenn(X_before, y_before):
    smotenn = SMOTEENN(random_state=42)
    X_after, y_after = smotenn.fit_resample(X_before, y_before)
    return X_after, y_after

def my_smotetomek(X_before, y_before):
    smotetomek = SMOTETomek(random_state=42)
    X_after, y_after = smotetomek.fit_resample(X_before, y_before)
    return X_after, y_after


# 3. Compile Preprocessing Pipeline

All of the preprocessing classes or functions defined earlier will be compiled in this step.

If you use sklearn to create preprocessing classes, you can list your preprocessing classes in the Pipeline object sequentially, and then fit and transform your data.

In [402]:
from sklearn.pipeline import Pipeline


pipe = Pipeline([
                 ("feature_imputer", FeatureImputer()),
                 ("duplicate_remover", DuplicateRemover()),
                 ("feature_selector", FeatureSelector()),
                 ("encoder", FeatureEncoder()),
                 ("feature_object_dropper", FeatureObjectDropper()),
                 ("feature_creator", FeatureCreator()),
                 ("feature_transformator", FeatureTransformator()),
                 ("feature_scaler", FeatureScaler()),
                 ])



In [403]:
train_set = pipe.fit_transform(train_set)
val_set = pipe.transform(val_set)

# 4. Modeling and Validation

Modelling is the process of building your own machine learning models to solve specific problems, or in this assignment context, predicting the target feature `label`. Validation is the process of evaluating your trained model using the validation set or cross-validation method and providing some metrics that can help you decide what to do in the next iteration of development.

In [404]:
X_train = train_set.drop(columns=["label"])
y_train = train_set['label']


In [405]:
X_val= val_set.drop(columns=["label"])
y_val = val_set['label']

In [406]:
X_train, y_train = my_smote(X_train, y_train)

# ? bawah tuh train buat si val_set (dipisahin takutny ada kenapa-kenapa pas digabung bareng si tes)
X_train_temp = X_train.copy()
y_train_temp = y_train.copy()


## A. KNN

In [407]:
# INI DRI LIBRARY
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=1)
knn.fit(X_train, y_train)
knn_pred = knn.predict(X_val)

In [408]:
from sklearn.metrics import accuracy_score, f1_score, recall_score, cohen_kappa_score, balanced_accuracy_score, classification_report
print(f'Accuracy: {accuracy_score(y_val, knn_pred)}')
print(f'F1 Score: {f1_score(y_val, knn_pred, average="weighted")}')
print(f'Recall Score: {recall_score(y_val, knn_pred, average="weighted")}')
print(f"Balanced Accuracy: {balanced_accuracy_score(y_val, knn_pred)}")
print("KNeighbors Classifier from libary")
print(classification_report(y_val, knn_pred))
print()

Accuracy: 0.9924148000427335
F1 Score: 0.9924287680819363
Recall Score: 0.9924148000427335
Balanced Accuracy: 0.9745741588288203
KNeighbors Classifier from libary
              precision    recall  f1-score   support

           0       0.95      0.95      0.95      2111
           1       1.00      1.00      1.00     25970

    accuracy                           0.99     28081
   macro avg       0.97      0.97      0.97     28081
weighted avg       0.99      0.99      0.99     28081




In [409]:
# ? new (jan hapus yang atas, buat referensi)
class KNearestNeighbors:
    def __init__(self, k=3, distance='euclidean', p=2):
        self.k = k
        self.distance = distance
        self.p = p
        self.X_train = None
        self.y_train = None
    
    def fit(self, X, y):
        self.X_train = np.array(X)
        self.y_train = np.array(y)

    def _compute_distance(self, x_new):
        if self.distance == 'euclidean':
            distances = np.sqrt(np.sum((self.X_train - x_new) ** 2, axis=1))
        elif self.distance == 'manhattan':
            distances = np.sum(np.abs(self.X_train - x_new), axis=1)
        elif self.distance == 'minkowski':
            distances = np.sum(np.abs(self.X_train - x_new) ** self.p, axis=1) ** (1 / self.p)
        else:
            raise ValueError("Pilih 'euclidean', 'manhattan', atau 'minkowski'.")
        
        return distances
    
    def predict(self, X):
        X = np.array(X)
        y_pred = []
        
        for x_new in X:
            distances = self._compute_distance(x_new)
            nearest_indices = np.argsort(distances)[:self.k]
            nearest_labels = self.y_train[nearest_indices]
            labels, counts = np.unique(nearest_labels, return_counts=True)
            majority_label = labels[np.argmax(counts)]
            
            y_pred.append(majority_label)
        
        return np.array(y_pred)

In [410]:
# ? buat val set
knn = KNearestNeighbors(k = 3)
knn.fit(X_train, y_train)
knn_pred = knn.predict(X_val)

In [411]:
print(f'Accuracy: {accuracy_score(y_val, knn_pred)}')
print(f'F1 Score: {f1_score(y_val, knn_pred, average="weighted")}')
print(f'Recall Score: {recall_score(y_val, knn_pred, average="weighted")}')
print(f"Balanced Accuracy: {balanced_accuracy_score(y_val, knn_pred)}")
print("KNeighbors Classifier from scratch")

print(classification_report(y_val, knn_pred))

Accuracy: 0.9910615718813433
F1 Score: 0.991148023940163
Recall Score: 0.9910615718813433
Balanced Accuracy: 0.9777593739961954
KNeighbors Classifier from scratch
              precision    recall  f1-score   support

           0       0.92      0.96      0.94      2111
           1       1.00      0.99      1.00     25970

    accuracy                           0.99     28081
   macro avg       0.96      0.98      0.97     28081
weighted avg       0.99      0.99      0.99     28081



## B. Naive Bayes

In [412]:
from sklearn.naive_bayes import GaussianNB


gnb_lib = GaussianNB()


gnb_lib.fit(X_train, y_train)


gnb_lib_pred = gnb_lib.predict(X_val)


print(f'Accuracy: {accuracy_score(y_val, gnb_lib_pred)}')
print(f'F1 Score: {f1_score(y_val, gnb_lib_pred, average="weighted")}')
print(f'Recall Score: {recall_score(y_val, gnb_lib_pred, average="weighted")}')
print(f"Balanced Accuracy: {balanced_accuracy_score(y_val, gnb_lib_pred)}")
print("Gaussian Naive Bayes with library")
print(classification_report(y_val, gnb_lib_pred))

Accuracy: 0.984544709946227
F1 Score: 0.9849065717648015
Recall Score: 0.984544709946227
Balanced Accuracy: 0.9692312413824427
Gaussian Naive Bayes with library
              precision    recall  f1-score   support

           0       0.86      0.95      0.90      2111
           1       1.00      0.99      0.99     25970

    accuracy                           0.98     28081
   macro avg       0.93      0.97      0.95     28081
weighted avg       0.99      0.98      0.98     28081



In [413]:
import numpy as np
import pandas as pd

class GaussianNaiveBayes:
    def __init__(self):
        self.classes_ = None
        self.priors_ = {}
        self.means_ = {}
        self.vars_ = {}
    
    def fit(self, X, y):
        if isinstance(X, pd.DataFrame):
            X = X.values
        if isinstance(y, pd.Series):
            y = y.values
        
        self.classes_ = np.unique(y)
        
        for cls in self.classes_:
            X_cls = X[y == cls]
            self.priors_[cls] = X_cls.shape[0] / X.shape[0]
            self.means_[cls] = np.mean(X_cls, axis=0)
            self.vars_[cls] = np.var(X_cls, axis=0) + 1e-9
    
    def _pdf(self, x, mean, var):
        coeff = 1.0 / np.sqrt(2.0 * np.pi * var)
        exponent = np.exp(- (x - mean)**2 / (2 * var))
        return coeff * exponent
    
    def _class_likelihood(self, x):

        posteriors = {}
        for cls in self.classes_:
            prior = np.log(self.priors_[cls])  
            pdf_vals = self._pdf(x, self.means_[cls], self.vars_[cls])
            pdf_vals_clipped = np.clip(pdf_vals, 1e-15, None)
            likelihoods = np.sum(np.log(pdf_vals_clipped))
            posterior = prior + likelihoods
            posteriors[cls] = posterior

        return posteriors
    
    def predict(self, X):

        if isinstance(X, pd.DataFrame):
            X = X.values
        
        y_pred = []
        for x in X:
            posteriors = self._class_likelihood(x)
            cls = max(posteriors, key=posteriors.get)
            y_pred.append(cls)
        return np.array(y_pred)


In [414]:
gnb = GaussianNaiveBayes()

In [415]:
gnb.fit(X_train,y_train)


In [416]:
gnb_pred = gnb.predict(X_val)

In [417]:
print(f'Accuracy: {accuracy_score(y_val, gnb_pred)}')
print(f'F1 Score: {f1_score(y_val, gnb_pred, average="weighted")}')
print(f'Recall Score: {recall_score(y_val, gnb_pred, average="weighted")}')
print(f"Balanced Accuracy: {balanced_accuracy_score(y_val, gnb_pred)}")
print("Gauss Naive Bayes from scratch")
print(classification_report(y_val, gnb_pred))
print()

Accuracy: 0.9855418254335672
F1 Score: 0.985809415371779
Recall Score: 0.9855418254335672
Balanced Accuracy: 0.9673767074825068
Gauss Naive Bayes from scratch
              precision    recall  f1-score   support

           0       0.87      0.95      0.91      2111
           1       1.00      0.99      0.99     25970

    accuracy                           0.99     28081
   macro avg       0.93      0.97      0.95     28081
weighted avg       0.99      0.99      0.99     28081




## C. Improvements (Optional)

- **Visualize the model evaluation result**

This will help you to understand the details more clearly about your model's performance. From the visualization, you can see clearly if your model is leaning towards a class than the others. (Hint: confusion matrix, ROC-AUC curve, etc.)

- **Explore the hyperparameters of your models**

Each models have their own hyperparameters. And each of the hyperparameter have different effects on the model behaviour. You can optimize the model performance by finding the good set of hyperparameters through a process called **hyperparameter tuning**. (Hint: Grid search, random search, bayesian optimization)

- **Cross-validation**

Cross-validation is a critical technique in machine learning and data science for evaluating and validating the performance of predictive models. It provides a more **robust** and **reliable** evaluation method compared to a hold-out (single train-test set) validation. Though, it requires more time and computing power because of how cross-validation works. (Hint: k-fold cross-validation, stratified k-fold cross-validation, etc.)

## D. Submission
To predict the test set target feature and submit the results to the kaggle competition platform, do the following:
1. Create a new pipeline instance identical to the first in Data Preprocessing
2. With the pipeline, apply `fit_transform` to the original training set before splitting, then only apply `transform` to the test set.
3. Retrain the model on the preprocessed training set
4. Predict the test set
5. Make sure the submission contains the `id` and `label` column.

Note: Adjust step 1 and 2 to your implementation of the preprocessing step if you don't use pipeline API from `sklearn`.

In [418]:
from sklearn.pipeline import Pipeline


pipe_submit = Pipeline([
                 ("feature_imputer", FeatureImputer()),
                 ("duplicate_remover", DuplicateRemover()),
                 ("feature_selector", FeatureSelector()),
                 ("encoder", FeatureEncoder()),
                 ("feature_object_dropper", FeatureObjectDropper()),
                 ("feature_creator", FeatureCreator()),
                 ("feature_transformator", FeatureTransformator()),
                 ("feature_scaler", FeatureScaler()),
                 ])

In [419]:
train_set_original = pipe_submit.fit_transform(train_set_original)
df_test = pipe_submit.transform(df_test)

In [420]:
X_train_submit = train_set_original.drop(columns=["label"])
y_train_submit = train_set_original['label']

In [421]:
X_train_submit, y_train_submit = my_smote(X_train_submit, y_train_submit)

In [422]:
knn = KNearestNeighbors(k = 3)
knn.fit(X_train_submit, y_train_submit)

submission = pd.read_csv('sample_submission.csv')
submission['label'] = knn.predict(df_test) 
submission.to_csv('submission_nn.csv', index=False)
submission.head()

,id,label
0,48,1
1,68,0
2,76,1
3,155,1
4,167,0


In [423]:
import pickle
pickle.dump(knn, open("knn.pkl", "wb"))


# 6. Error Analysis

Based on all the process you have done until the modeling and evaluation step, write an analysis to support each steps you have taken to solve this problem. Write the analysis using the markdown block. Some questions that may help you in writing the analysis:

- Does my model perform better in predicting one class than the other? If so, why is that?
- To each models I have tried, which performs the best and what could be the reason?
- Is it better for me to impute or drop the missing data? Why?
- Does feature scaling help improve my model performance?
- etc...

`Provide your analysis here`